<a href="https://colab.research.google.com/github/EkaterinaVZ/hackathon_2023/blob/main/part3__document_verification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Хакатон_2023

* загружаем ранее подготовленный датасет с собранной информацией из json + сущности из docx и датасет с текстами из документов<br>
* выделяем необходимые данные для сравнения документов<br>
* проверяем новые документы на наличие в них ошибок

Имеющаяся информация:
* id,
* 'ORG' - название организации,
* 'signatory' - подписант
* 'date_sig' - дата подписания,
* 'position_of_signatory' - должность подписанта,
* 'title' - название документа,
* 'Task1.task_text' - текст задачи,
* 'Task1.task_responsibles_people' - ответственный за задачу,
* 'Task1.task_responsibles_groups' - ответственный отдел за задачу,
* 'Task1.task_deadline' - срок выполнения задачи, (и так по каждой задаче),
* 'Global_supervisor' - главный ответственный,
* 'Global_deadline' - общий срок выполнения

### Ипортируем необходимые библиотеки

In [ ]:
! pip install python-docx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import zipfile

import docx
from termcolor import colored

import re

import json
import pandas as pd
import numpy as np

In [ ]:
!python -m spacy download ru_core_news_lg

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-01-20 16:49:23.077719: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 MB 2.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')


In [ ]:
import spacy
nlp = spacy.load('ru_core_news_lg')

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


# Загружаем подготовленные ранее данные

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df1 = pd.read_csv("/content/drive/MyDrive/Хакатон_2023/markup_18.01.2023.csv")

In [ ]:
df1.head(3)

,id,Signatory,Organization,Task1.task_text,Date_sig,Title,Task1.task_responsibles_people,Task1.task_responsibles_groups,Task1.task_deadline,Task2.task_text,...,responsibles_people_Task4.,responsibles_people_Task5.,responsibles_people_Task6.,responsibles_people_Task7.,responsibles_people_Task8.,responsibles_people_Task9.,responsibles_people_respon,Nasmes_global_supervisor,Names_global_supervisor,Posts_global_supervisors
0,0,и.о. генерального директора Сердитый А.И.,"ООО ""Мегасофт""",Закупить книги. Срок завершения: 19 апр 15.,11.4.2013,Приказ,Ответственность за исполнение распоряжения воз...,Административно-хозяйственный отдел,"['19 апр 15', 1429383600.0]",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['Кравцову И.Е.'],['специалиста первой категории']
1,1,и.о. генерального директора Сердитый А.И.,"ООО ""Мегасофт""",Провести анализ поставщиков для замены вышедше...,3.6.2013,Приказ,Ответственность за исполнение распоряжения воз...,Служба ИТ,"['24 октября 12', 1351015200.0]",Выполнить ремонт кабеля сети передачи данных м...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Кима Юрия Ивановича', 'Сергеева Андрея', 'Ко...","['специалиста третьей категории', 'системного ..."
2,2,генеральный директор Созонов С.В.,"ООО ""Мегасофт""",Провести рефакторинг программного обеспечения ...,8.10.2014,Приказ,Контроль над исполнением распоряжения оставить...,Отдел разработки систем,"['24.12.20', 1608750000.0]",Провести рефакторинг программного обеспечения ...,...,технического писателя,специалистом третьей категории,специалиста третьей категории,NaN,NaN,NaN,NaN,NaN,"['Абаимовым Симоном', 'Ответственным', 'Абаимо...","['тимлидами проектной команды', 'тимлид проект..."


создадим отдельный датафрейм с перечнем департаментов, сотрудников и их должностей

In [ ]:
triplets = [
    ['Task1.task_responsibles_groups', 'responsibles_people_Task1.', 'executor_Task1.',],
    ['Task1.task_responsibles_groups', 'responsibles_people_Task2.', 'executor_Task2.',],
    ['Task1.task_responsibles_groups', 'responsibles_people_Task3.', 'executor_Task3.',],
    ['Task1.task_responsibles_groups', 'responsibles_people_Task4.', 'executor_Task4.',],
    ['Task1.task_responsibles_groups', 'responsibles_people_Task5.', 'executor_Task5.',],
    ['Task1.task_responsibles_groups', 'responsibles_people_Task6.', 'executor_Task6.',],
    ['Task1.task_responsibles_groups', 'responsibles_people_Task7.', 'executor_Task7.',],
]
dataframes = [df1[columns] for columns in triplets]

for dataframe in dataframes:
    dataframe.columns = ["Департамент", "Должность", "ФИО"]

df = pd.concat(dataframes, ignore_index=True)

In [ ]:
df.head(5)

,Департамент,Должность,ФИО
0,Административно-хозяйственный отдел,специалиста первой категории,Кравцову И.Е.
1,Служба ИТ,специалиста третьей категории,Кима Юрия Ивановича
2,Отдел разработки систем,тимлидами проектной команды,Абаимовым Симоном
3,Бухгалтерия,NaN,NaN
4,Бухгалтерия,NaN,NaN


подготовим функции для приведения текста в единый формат для дальнейшего сравнения

In [ ]:
def clear_text(text: str) -> str:
    """
    Subfunction to clear text from 'trash'
    
    :param text: Source text
    
    :return: text - Cleared text
    """
    mentions = r'@[A-Za-z0-9_-]+'
    urls = r'https?://[A-Za-z0-9./]+'
    punct = r'[.",#!$%\^&\*;:{}=\-_`~()]'

    for regx in [mentions, urls, punct]:
        text = re.sub(regx, ' ', text)
        text=re.sub('\s+',' ', text) 
        
    text = text.strip()

    return text

In [ ]:
def clear_all_texts(x):
    txt =""
    doc = nlp(x)
    for entity in doc:
        res = entity.lemma_
        txt += (res + " ")
    return txt.rstrip()

удалим строки с пропусками

In [ ]:
df = df.dropna(axis=0)
df.index = range(len(df))
df

,Департамент,Должность,ФИО
0,Административно-хозяйственный отдел,специалиста первой категории,Кравцову И.Е.
1,Служба ИТ,специалиста третьей категории,Кима Юрия Ивановича
2,Отдел разработки систем,тимлидами проектной команды,Абаимовым Симоном
3,Бухгалтерия,специалиста второй категории,Смирнову О.А.
4,Отдел разработки систем,специалиста третьей категории,Андреева С.Е.
...,...,...,...
35330,Отдел разработки систем,программиста,Абашкина Ореста
35331,Административно-хозяйственный отдел,специалиста первой категории,Кравцову И.Е.
35332,Отдел тестирования,специалиста третьей категории,Спецова
35333,Служба ИТ,мастера по ремонту компьютеров,Абалакина Михаила


In [ ]:
print(f'Количество департаментов: {df["Департамент"].nunique()}')
print()
print(f'Список департаментов: {(df["Департамент"].unique())}')
print()
print(f'Количество сотрудников: {df["ФИО"].nunique()}')

Количество департаментов: 8

Список департаментов: ['Административно-хозяйственный отдел' 'Служба ИТ'
 'Отдел разработки систем' 'Бухгалтерия' 'Отдел проектирования'
 'Отдел архитектуры' 'Отдел эксплуатации' 'Отдел тестирования']

Количество сотрудников: 58


посмотрим на количество упоминаний департаментов и сотрудников в документах

In [ ]:
df.groupby('Департамент').count()

,Должность,ФИО
Департамент,,
Административно-хозяйственный отдел,7470,7470
Бухгалтерия,7800,7800
Отдел архитектуры,1544,1544
Отдел проектирования,1501,1501
Отдел разработки систем,7189,7189
Отдел тестирования,1586,1586
Отдел эксплуатации,2042,2042
Служба ИТ,6203,6203


In [ ]:
df.groupby('Должность').count()

,Департамент,ФИО
Должность,,
администратора баз,738,738
администратора баз данных,756,756
администратора вычислительной,407,407
администратора вычислительной сети,416,416
администратором вычислительной,93,93
архитектора программного обеспечения,519,519
главного бухгалтера,1893,1893
мастера по ремонту компьютеров,800,800
мастером по ремонту компьютеров,107,107


In [ ]:
df.groupby('ФИО').count()

,Департамент,Должность
ФИО,,
Абаимова Симона,697,697
Абаимовым Симоном,83,83
Абакова,129,129
Абакова Алексея,108,108
Абакулова Петра,841,841
Абакумова Глеба,244,244
Абалакина Михаила,800,800
Абалакиным Михаилом,107,107
Абалакова Севастьяна,406,406


удалим дубликаты

In [ ]:
df_unique = df.drop_duplicates()
df_unique.shape

(63, 3)

приведем текст к единообразному виду

In [ ]:
for i in df_unique.columns:
    df_unique[i] = df_unique[i].apply(lambda x: clear_all_texts(clear_text(str(x))) if x else None)

<ipython-input-18-2f09104191af>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique[i] = df_unique[i].apply(lambda x: clear_all_texts(clear_text(str(x))) if x else None)


посмотрим на распределние сотрудников по депарматентам

In [ ]:
df_unique.groupby('Департамент').count()

,Должность,ФИО
Департамент,,
административно хозяйственный отдел,4,4
бухгалтерия,5,5
отдел архитектура,9,9
отдел проектирование,5,5
отдел разработка система,13,13
отдел тестирование,5,5
отдел эксплуатация,7,7
служба ит,15,15


In [ ]:
departments = df_unique['Департамент'].unique()

In [ ]:
for department in departments:
    display(df_unique[df_unique['Департамент'] == department])

,Департамент,Должность,ФИО
0,административно хозяйственный отдел,специалист первый категория,кравцов и е
11,административно хозяйственный отдел,руководитель административно хозяйственный,с с галкин
19,административно хозяйственный отдел,специалист первый категория,сергеев
51,административно хозяйственный отдел,специалист второй категория,поспелов


,Департамент,Должность,ФИО
1,служба ит,специалист третий категория,ким юрий иванович
6,служба ит,администратор база,абакулова пётр
29,служба ит,администратор вычислительный сеть,абалдуева олег
38,служба ит,специалист второй категория,никитин ирина петрович
60,служба ит,мастер по ремонт компьютер,абалакина михаил
65,служба ит,мастер по ремонт компьютер,абалакиным михаил
69,служба ит,администратор база данных,абакулова пётр
98,служба ит,администратор вычислительный,абалдуева олег
105,служба ит,специалист первый категория,александр иванович
124,служба ит,специалист третий категория,ким юрий иванович


,Департамент,Должность,ФИО
2,отдел разработка система,тимлидами проектный команда,абаимовым симон
4,отдел разработка система,специалист третий категория,андреев с е
5,отдел разработка система,специалист второй категория,попов а а
18,отдел разработка система,программист,абашкин орест
35,отдел разработка система,специалист первый категория,никишин с е
36,отдел разработка система,тимлид проектный команда,абаимов симона
39,отдел разработка система,администратор база,абашичева алексей
57,отдел разработка система,технический писатель,абалкин зураба
67,отдел разработка система,специалист третий категория,андреевым с е
123,отдел разработка система,администратор база данных,абашичева алексей


,Департамент,Должность,ФИО
3,бухгалтерия,специалист второй категория,смирнов о а
12,бухгалтерия,специалист первый категория,миронов т п
14,бухгалтерия,главный бухгалтер,семёнов м с
20,бухгалтерия,специалист треть категория,александров а с
73,бухгалтерия,специалист треть категория,александров а с


,Департамент,Должность,ФИО
7,отдел проектирование,специалист второй категория,попов и и
15,отдел проектирование,специалист первый категория,семёнов и а
16,отдел проектирование,специалист второй категория,попов и и
170,отдел проектирование,специалист третий категория,коптелова
348,отдел проектирование,специалист третий категория,коптеловым е в


,Департамент,Должность,ФИО
9,отдел архитектура,специалист третий категория,слесарев а в
48,отдел архитектура,технический писатель,абакумов глеб
116,отдел архитектура,специалист первый категория,миронов к а
133,отдел архитектура,архитектор программный обеспечение,абакова алексей
137,отдел архитектура,архитектор программный обеспечение,абакова
184,отдел архитектура,архитектор программный обеспечение,абатуров борис
303,отдел архитектура,специалист второй категория,симонов н с
371,отдел архитектура,специалист третий категория,слесаревым а в
1767,отдел архитектура,специалист первый категория,мироновым к а


,Департамент,Должность,ФИО
21,отдел эксплуатация,специалист второй категория,ведерников и с
24,отдел эксплуатация,специалист третий категория,жуков
61,отдел эксплуатация,специалист первый категория,иванов а а
126,отдел эксплуатация,специалист первый категория,иванов а а
155,отдел эксплуатация,руководитель отдел эксплуатация,точнова с о
161,отдел эксплуатация,специалист третий категория,жуков а в
2554,отдел эксплуатация,специалист второй категория,ведерников и с


,Департамент,Должность,ФИО
106,отдел тестирование,специалист второй категория,анненков и с
178,отдел тестирование,специалист второй категория,анненков и с
240,отдел тестирование,специалист первый категория,янина и а
249,отдел тестирование,специалист третий категория,спецова
706,отдел тестирование,специалист первый категория,яниным и а


# Проверка новых документов на наличие ошибок

подгружаем искусственно созданные файлы, содержащие ошибки

In [67]:
file_names = []
all_text = []

path_normalized = os.path.normpath("/content/drive/MyDrive/Хакатон_2023/test")
for dirpath, dirnames, filenames in os.walk(path_normalized):
    for file in filenames:
        file_names.append(int(file.rstrip('.docx')))
        text_doc = []
        doc = docx.Document(os.path.normpath(path_normalized + "/" + file))
        # собираем документ в список по абзацам и удаляем пустые абзацы
        for par in doc.paragraphs:
            if par.text != "":
                text = clear_text(par.text)
                text_doc.append(text.replace("\n", ""))
        all_text.append(text_doc)
        
raw_df = pd.DataFrame({"id": file_names, "text": all_text}, columns=["id", "text"])
# raw_df["number_of_paragraphs"] = raw_df["text"].str.len()
raw_df.set_index('id', inplace=True)
raw_df.sort_index(inplace=True) # сортируем по индеку=id 
print(raw_df.tail(5))

                                                    text
id                                                      
17563  [ООО Мегасофт, Приказ, В связи с подготовкой п...
18344  [ООО Мегасофт, Приказ, В связи с окончанием пр...
19196  [ООО Мегасофт, Приказ, В связи с окончанием пр...
19877  [ООО Мегасофт, Приказ, В связи с подготовкой п...
19969  [ООО Мегафон, Приказ, В связи с подготовкой пр...


In [ ]:
general_information = pd.DataFrame(raw_df['text'].to_list()) # создаем DataFrame
general_information = general_information.T
general_information.head(5)

приведем текст к единообразному виду

In [ ]:
for i in general_information.columns: 
    general_information[i] = general_information[i].apply(lambda x: clear_all_texts(x) if x else None)
 

In [ ]:
general_information.head(4)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,ооо морковь,ооо максидом,ооо стол и стул,ооо мегасофт,ооо мегасофт,ооо мегасофт,ооо мегасофт,ооо мегасофт 11,ооо мегасофт,ооо мегасофт,ооо мегасофт,ооо ромашка мегасофт,ооо мегасофт,ооо мегасофт,ооо мегасофт,ооо мегасофт,ооо мегасофт,ооо мегасофт,ооо мегасофт,ооо мегафон
1,приказ,приказ,приказ,приказ,приказ,приказ,приказ,приказ,приказ,приказ,приказ,приказ,приказ,приказ,приказ,приказ,приказ,приказ,приказ,приказ
2,в связь с окончание проект внедрение система з...,в связь с подготовка производственный мероприя...,для организация мероприятие день заказчик прик...,в связь с окончание проект внедрение система з...,в связь с окончание проект внедрение система з...,для организация мероприятие день заказчик прик...,в связь с окончание проект внедрение система з...,в связь с окончание проект внедрение система з...,для организация мероприятие день заказчик прик...,в связь с окончание проект внедрение система з...,для организация мероприятие день заказчик прик...,в связь с окончание проект внедрение система з...,для организация мероприятие день заказчик прик...,в связь с окончание проект внедрение система з...,в связь с окончание проект внедрение система з...,в связь с подготовка производственный мероприя...,в связь с окончание проект внедрение система з...,в связь с окончание проект внедрение система з...,в связь с подготовка производственный мероприя...,в связь с подготовка производственный мероприя...
3,a добавить дополнительный кролик в вольер прив...,a провести полив цвет не реже 1 раз в день опр...,a организовать хранение на офисный склад обору...,1 провести срочный ремонт и устранение неполад...,a провести рефакторинг программный обеспечение...,○ закупить необходимый количество клавиатура и...,○ подготовить финансовый отчётность приступить...,1 написать школьный сочинение за 11 класс в ра...,i установить программный обеспечение ms word н...,i добавить дополнительный стул в столовая для ...,i провести тестирование разработать модуль сро...,i уточнить список сотрудник иметь право на бес...,a предоставить сотрудник справка 2 ндфл опреде...,i разработать модуль обработка информация в си...,a выполнить стресс тестирование и система и пр...,1 написать документация для разработать функци...,i выполнить покраска стена приступить к исполн...,● устранить получить замечание налоговый служб...,○ подготовить бухгалтерский баланс выполнить д...,1 установить дополнительный шкаф в гардероб пр...


In [ ]:
general_information[0].unique()

array(['ооо морковь', 'приказ',
       'в связь с окончание проект внедрение система знание поручать обеспечить выполнение задача',
       'a добавить дополнительный кролик в вольер привести вышеизложенный поручение в исполнение к 11 09 15',
       'за контроль над исполнение распоряжение назначить ответственный старший кроликовод веточкина к к',
       'генеральный директор созонов с в 26 июль 15', None], dtype=object)

### Собираем данные из исходного датасета

In [ ]:
organizations = [clear_all_texts(clear_text(i)) for i in df1["Organization"].unique()]
organizations # имеющиеся названия организаций

['ооо мегасофт']

In [ ]:
signatories = [clear_all_texts(clear_text(i)) for i in df1["Signatory"].unique()]
signatories # имеющие право подписи в организации

['и о генеральный директор сердитый а и',
 'генеральный директор созонов с в',
 'и о генеральный директор кравцов с с']

In [ ]:
titles = [clear_all_texts(clear_text(i)) for i in df1["Title"].unique()]
titles # уникадьные названия документов

['приказ']

создаем первую часть датафрейма, в котором будут отображалься результаты проверки файлов на ошибки

In [ ]:
rslt_df = pd.DataFrame(index=["Организция", "Тип_документа", "Подписант"],columns=range(len(general_information.columns)))

In [ ]:
rslt_df.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Название_организц,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Название_док,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Подписант,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


функция для проверки совпадения названий организции, типов документов, подписантов

In [ ]:
def get_exist(x, num, name_list, raw_name):
  for org in name_list:
    if org in x:
      rslt_df[num][raw_name] = "OK"
      break

проверяем название организации

In [ ]:
for num, i in enumerate(general_information.columns):
    general_information[i].apply(lambda x: get_exist(x, num, organizations, 0) if x else None)


проверяем название документа

In [ ]:
for num, i in enumerate(general_information.columns):
    general_information[i].apply(lambda x: get_exist(x, num, titles, 1) if x else None)


проверяем название подписанта

In [ ]:
for num, i in enumerate(general_information.columns):
    general_information[i].apply(lambda x: get_exist(x, num, signatories, 2) if x else None)


In [ ]:
rslt_df = rslt_df.replace(np.NaN, 'NOK')
rslt_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Название_организц,NOK,NOK,NOK,OK,OK,OK,OK,OK,OK,OK,OK,NOK,OK,OK,OK,OK,OK,OK,OK,NOK
Название_док,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK
Подписант,OK,NOK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,NOK


Названия столбцов - это проверяемые документы (№ 0, 1, 2, 3 и т.д.).

Интерпретация результатов проверки: если в стоблце отсутствуют значения NOK (присутствуют только OK), то документ считается верным

### Проверям наличие сотрудников по задачам, их должности и отделы в сооответсвии со структурой предприятия

создаем 2 часть датафрейм, в котором будут отображалься результаты проверки файлов на ошибки

In [ ]:
rslt_df2 = pd.DataFrame(index=range(1, len(general_information)),columns=range(len(general_information.columns)))
rslt_df2.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
nlp = spacy.load('ru_core_news_lg')

создаем функцию для проверки ошибок в именах сотрудников, должностях, департаментах

In [ ]:
def check_person(x):
    # return "OK"
    persons = []
    doc = nlp(x)

    for entity in doc.ents:
        # print(entity.text)
        if entity.label_ == "PER":
            # print(entity.text)
            try:
              loc_df = df_unique[df_unique["ФИО"] == entity.text]

              if loc_df.iloc[0]["ФИО"]== entity.text:
                    return "OK"
            except:
                  return "NOK"
            
            # try:
            #     if loc_df.iloc[0]["ФИО"] == entity.text:
            #         nlp = spacy.load("/content/drive/MyDrive/Хакатон_2023/spacy/ner_group1")
            #         doc = nlp(x)
            #         for entity in doc.ents:
            #             if entity.label_ == "GROUP":
            #                 if loc_df.iloc[0]["Должность"] == entity.text:
            #                     return "OK"
            # except:
            #     return "NOK"
            # try:
            #     if loc_df.iloc[0]["ФИО"] == entity.text:

            #         nlp = spacy.load("/content/drive/MyDrive/Хакатон_2023/spacy/ner_post1")
            #         doc = nlp(x)
            #         for entity in doc.ents:

            #             if entity.label_ == "POST":

            #                 try:
            #                     print(entity.text)
            #                     if loc_df.iloc[0]["Должность"] == entity.text:
            #                         nlp = spacy.load("/content/drive/MyDrive/Хакатон_2023/spacy/ner_group1")

            #                         doc = nlp(x)
            #                         for entity in doc.ents:
            #                             if entity.label_ == "GROUP":
            #                                 return "OK"
            #                 except:
            #                     return "NOK"
            # except:
            #     return "NOK"

In [ ]:
for num, i in enumerate(general_information.columns):
    
    rslt_df2[num] = general_information[i].apply(lambda x: check_person(x) if x else None) 

### Результаты проверки документов на наличие ошибок

Названия столбцов - это проверяемые документы (№ 0, 1, 2, 3 и т.д.).

Названия строк - это задачи в документах.

Интерпретация результатов проверки: 
если в стоблце отсутствуют значения NOK (присутствуют только None и OK), то документ считается верным

In [ ]:
df_res = rslt_df.append(rslt_df2)
df_res

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Название_организц,NOK,NOK,NOK,OK,OK,OK,OK,OK,OK,OK,OK,NOK,OK,OK,OK,OK,OK,OK,OK,NOK
Название_док,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK
Подписант,OK,NOK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,OK,NOK
1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,None,NOK,NOK,None,None,None,NOK,None,NOK,NOK,OK,None,NOK,None,NOK,NOK,NOK,NOK,NOK,NOK
5,NOK,None,None,None,None,None,NOK,None,None,None,NOK,None,None,None,None,None,None,None,None,None
6,None,NOK,NOK,None,None,None,None,NOK,OK,None,None,None,NOK,None,OK,NOK,NOK,NOK,NOK,NOK
7,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


### Вывод:<br>
после обучения spacy поиску новых сущностей (должности, департамента, задач, подписанта) были проведены эксперименты на искусственно созданных документах с ошибками(см. ниже), в результате которых была сформирована результирующая таблица, из которой стало понятно, что необходимо доработать нахождение сущностей с помощью spacy или применить более сложные модели. Также планировалось реализовать поиск опечаток/ошибок.

#### эта часть использовалась для тестирования гипотез и ждет доработки

In [ ]:
# def verify_per(x):
#        persons = []
#        doc = nlp(x)
#        for entity in doc.ents:
#           if entity.label_ == "PER":
#             persons.append(entity.text)
#        return persons

In [ ]:
# new_df["persons"] = new_df["clear_text"].apply(lambda x: verify_per(x))

In [ ]:
# def get_info(x):
#   all = []
#   doc = nlp(x)
#   for ent in doc.ents:
#     print(ent)
#     all.append(ent.text)
#     # print(all)
#   return all

In [ ]:
# nlp = spacy.load("/content/drive/MyDrive/Хакатон_2023/spacy/ner_group1")

In [ ]:
# general_information["group"] = general_information["clear_text"].apply(lambda x: get_info(x))

In [ ]:
# nlp = spacy.load("/content/drive/MyDrive/Хакатон_2023/spacy/ner_post1")

In [ ]:
# general_information["post"] = general_information["clear_text"].apply(lambda x: get_info(x))

In [ ]:
# nlp = spacy.load("/content/drive/MyDrive/Хакатон_2023/spacy/ner_title3")

In [ ]:
# general_information["title"] = general_information["clear_text"].apply(lambda x: get_info(x))

In [ ]:
# nlp = spacy.load("/content/drive/MyDrive/Хакатон_2023/spacy/ner_sig4")

In [ ]:
# general_information["signatory"] = general_information["clear_text"].apply(lambda x: get_info(x))

In [ ]:
# nlp = spacy.load("/content/drive/MyDrive/Хакатон_2023/spacy/ner_tasks5")

In [ ]:
# general_information["tasks"] = general_information["clear_text"].apply(lambda x: get_info(x))

In [ ]:
# nlp = spacy.load("/content/drive/MyDrive/Хакатон_2023/spacy/ner_group1")

In [ ]:
# result = get_info(" первый категория административный - хозяйственный отдел сергеев ален ивановну b. подгот")
# result

In [ ]:
# nlp = spacy.load("/content/drive/MyDrive/Хакатон_2023/spacy/ner_emp6")

In [ ]:
# general_information["mpe"] = general_information["clear_text"].apply(lambda x: get_info(x))

In [ ]:
# new_df.drop(columns="verification_org", inplace=True)

In [ ]:
# new_df